# 102Flowers Image Classifier

This is the main notebook for the project. See the associated report (WIP) for more information.

### Imports

In [ ]:
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torchvision import datasets, transforms, models

import numpy as np
import matplotlib.pyplot as plt
import scipy

from IPython.display import clear_output

### Hyperparameters

In [ ]:
# TODO: Set hyperparameters.
DEFAULT_BATCH_SIZE = 20
TRAINING_BATCH_SIZE = DEFAULT_BATCH_SIZE
VALIDATION_BATCH_SIZE = DEFAULT_BATCH_SIZE
TESTING_BATCH_SIZE = DEFAULT_BATCH_SIZE
TESTING_BATCH_COUNT = 51
EPOCHS = 100
LEARNING_RATE = 0.0001
IMAGE_CROP_SIZE = 128
TRAINING_PLOT = True
DROPOUT_P = 0.2

### Device

In [ ]:
# Default to CPU
DEVICE = torch.device("cpu")

# Switch to GPU if available
if torch.cuda.is_available():
	print(f"Found {torch.cuda.device_count()} GPUs. Using cuda:0.")
	DEVICE = torch.device("cuda:0")
else:
	print("No GPUs found, using CPU.")

### Load Dataset

In [ ]:
training_data = datasets.Flowers102(
    root = "data",
    split = "train",
    transform=transforms.Compose([
        transforms.Resize(IMAGE_CROP_SIZE),
        transforms.CenterCrop(IMAGE_CROP_SIZE),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.ToTensor()
    ]),
    download=True
)

validation_data = datasets.Flowers102(
    root = "data",
    split = "val",
    transform=transforms.Compose([
        transforms.Resize(IMAGE_CROP_SIZE),
        transforms.CenterCrop(IMAGE_CROP_SIZE),
        transforms.ToTensor()
    ]),
    download=True
)

testing_data = datasets.Flowers102(
    root = "data",
    split = "test",
    transform=transforms.Compose([
        transforms.Resize(IMAGE_CROP_SIZE),
        transforms.CenterCrop(IMAGE_CROP_SIZE),
        transforms.ToTensor()
    ]),
    download=True
)

### DataLoaders

In [ ]:
training_dataloader = DataLoader(training_data, batch_size=TRAINING_BATCH_SIZE, shuffle=True)
validation_dataloader = DataLoader(validation_data, batch_size=VALIDATION_BATCH_SIZE, shuffle=True)
testing_dataloader = DataLoader(testing_data, batch_size=TESTING_BATCH_SIZE, shuffle=True)

## Model

In [ ]:
#classifications = F.one_hot(torch.tensor([e for e in range(0,102)]), num_classes=102)

In [ ]:
class F102Classifier(nn.Module):
    
	def __init__(self):
		super(F102Classifier, self).__init__()
		
		self.pool = nn.MaxPool2d(2, 2)
		self.dropout = nn.Dropout(p=DROPOUT_P)
		self.conv1 = nn.Conv2d(3, 6, 3)
		self.bn1 = nn.BatchNorm2d(6)
		self.conv2 = nn.Conv2d(6, 12, 3)
		self.bn2 = nn.BatchNorm2d(12)
		self.conv3 = nn.Conv2d(12, 24, 3)
		self.bn3 = nn.BatchNorm2d(24)
		self.fc1 = nn.Linear(83544, 1020)
		self.fc2 = nn.Linear(1020, 102)

	def forward(self, x):

		x = self.pool(F.relu(self.conv1(x)))
		#x = F.relu(self.conv1(x))
		x = self.bn2(F.relu(self.conv2(x)))
		x = self.bn3(F.relu(self.conv3(x)))
		x = torch.flatten(x)
		x = x.view(TRAINING_BATCH_SIZE, -1)
		x = F.relu(self.fc1(x))
		x = self.dropout(x)
		x = self.fc2(x)
		#print(x.size())
		return x

net = F102Classifier()
net = net.to(DEVICE)

### Loss Function & Optimiser

In [ ]:
loss_function = nn.CrossEntropyLoss().to(DEVICE)
optimiser = optim.Adam(net.parameters(), lr=LEARNING_RATE)

## Validation, Testing, Training Functions

### Validation, Testing Functions

In [ ]:
def validate(model:F102Classifier=net, dataloader:DataLoader=validation_dataloader, loss_fn=loss_function, batches=TESTING_BATCH_COUNT, print_type="Validation"):
    model.eval()
    test_loss, correct = 0, 0
    for batch, (inputs, labels) in enumerate(dataloader):
        if batch == batches:
            break
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        output = model(inputs)
        test_loss += loss_fn(output, labels).item()
        pred = torch.tensor([torch.argmax(o)+1 for o in output]).to(DEVICE)
        correct += pred.eq(labels.view_as(pred)).sum().item()
    test_loss /= batches
    correct /= batches
    print(f"{print_type} Accuracy: {correct*100:.1f}%\nAverage Loss for Test: {test_loss:.3f}")
    return test_loss, correct

def test(model:F102Classifier=net, dataloader:DataLoader=testing_dataloader, loss_fn=loss_function, batches=TESTING_BATCH_COUNT, print_type="Testing"):
    return validate(dataloader=dataloader, batches=TESTING_BATCH_COUNT, print_type=print_type)

### Training Function

In [ ]:
def train(model:F102Classifier=net, dataloader:DataLoader=training_dataloader, loss_fn=loss_function, optimiser=optimiser, epochs:int=EPOCHS, validate_every:int=1, training_fit_every:int=5, plot:bool=TRAINING_PLOT):
	print("Started Training")
	model.train()
	epoch_loss = 0
	loss_record = list([0])
	validation_accuracy_record = list([(0,0)])
	training_accuracy_record = list([(0,0)])
	validation_loss_record = list([(0,0)])
	training_fit_loss_record = list([(0,0)])
	for i in range(0, epochs):
		for batch, (inputs, labels) in enumerate(dataloader):
			inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)

			outputs = model(inputs)
			#outputs = torch.tensor([torch.argmax(o)+1 for o in outputs]).to(DEVICE)
			
			# LOSS
			loss = loss_fn(outputs, labels)
			epoch_loss += loss.item()

			# BACKPROP
			optimiser.zero_grad()
			loss.backward()
			optimiser.step()
			#print(batch)
			#print(loss_record)
			#print(accuracy_record)

			if (batch % len(dataloader) == 0):
				clear_output(wait=True)

				if ((i + 1) % validate_every == 0):
					test_loss, accuracy = validate()
					validation_accuracy_record.append((i+1,accuracy*100))
					validation_loss_record.append((i+1,test_loss))

				if ((i + 1) % training_fit_every == 0):
					test_loss, accuracy = validate(dataloader=training_dataloader, print_type="Training")
					training_accuracy_record.append((i+1,accuracy*100))
					training_fit_loss_record.append((i+1,test_loss))

				epoch_loss = epoch_loss/len(dataloader)
				loss_record.append(epoch_loss)
				status = f"Epoch: {i+1}/{epochs}\nBatch: {batch+1}/{len(dataloader)}\nMean Loss for Epoch: {epoch_loss:.4f}"
				epoch_loss = 0.0
				print(status)

				if plot:
					fig, ax1 = plt.subplots()
					ax1.plot(loss_record, color='limegreen', label='Epoch Loss')
					ax1.plot([e[0] for e in validation_loss_record], [a[1] for a in validation_loss_record], color='orange', label='Validation Set Loss')
					ax1.plot([e[0] for e in training_fit_loss_record], [a[1] for a in training_fit_loss_record], color='blue', label='Training Set Loss')
					plt.xlabel('Mean Loss for Epoch')
					plt.ylabel('Loss')
					plt.text(0, -0.1, status, ha='left', va='top', transform=plt.gca().transAxes)
					plt.gca().xaxis.set_major_locator(plt.MaxNLocator(integer=True))
					plt.gca().set_xlim(left=1)
					plt.gca().set_ylim(bottom=0)
					plt.gca().set_ylim(top=max(loss_record)*1.1)
					#plt.legend()
					# add secondary y axis for accuracy
					ax2 = ax1.twinx()
					ax2.plot([e[0] for e in validation_accuracy_record], [a[1] for a in validation_accuracy_record], color='sandybrown', label='Validation Set Accuracy (%)')
					ax2.plot([e[0] for e in training_accuracy_record], [a[1] for a in training_accuracy_record], color='slateblue', label='Training Set Accuracy (%)')
					plt.ylabel('Accuracy (%)')
					plt.gca().set_ylim(bottom=0)
					plt.gca().set_ylim(top=100)
					plt.gca().set_xlim(left=1)
					#plt.gca().set_xlim(right=len(loss_record)-1)
					plt.gca().xaxis.set_major_locator(plt.MaxNLocator(integer=True))
					# add legend
					li1, la1 = ax1.get_legend_handles_labels()
					li2, la2 = ax2.get_legend_handles_labels()
					lis, las = li1+li2, la1+la2
					plt.legend(lis, las)
					# draw gridlines
					plt.grid(which='both')
					plt.show()

	print("Finished Training")
	return loss_record, validation_accuracy_record


# Run Model

In [ ]:
train()

In [ ]:
### ARGMAX TESTING ###
a = torch.randn(32,102)
#type(a)
a = torch.tensor([torch.argmax(e) + 1 for e in a])
a

In [ ]:
### PLOT TESTING ###
loss_record = [61,42,9,1,2,3,4,3,2,1,2,3,4,1,2,3,4,5]
validation_accuracy_record = [(3, 30), (4, 40), (6, 40), (8, 20), (9, 100), (18,50)]
status = "bababooey"
plt.plot(loss_record)
plt.xlabel('Epoch')
plt.ylabel('Mean Loss for Epoch')
plt.text(0, 1, status, ha='left', va='top', transform=plt.gca().transAxes)
plt.gca().xaxis.set_major_locator(plt.MaxNLocator(integer=True))
plt.gca().set_xlim(left=0)
plt.gca().set_ylim(bottom=0)
plt.gca().set_ylim(top=max(loss_record)*1.1)
# add secondary y axis for accuracy
plt.twinx()
plt.plot([e[0] for e in validation_accuracy_record], [a[1] for a in validation_accuracy_record], color='orange')
plt.ylabel('Accuracy (%)')
plt.gca().set_ylim(bottom=0)
plt.gca().set_ylim(top=100)
plt.gca().set_xlim(left=0)
#plt.gca().set_xlim(right=len(loss_record)-1)
plt.gca().xaxis.set_major_locator(plt.MaxNLocator(integer=True))
plt.show()

### Save Model

In [ ]:
save_path = "./models/classifier.pth"
torch.save(net.state_dict(), save_path)